##JSON File to Table Format

## Zheng Zeng-Mar 10.2025


---





### 1. Read JSON file into dataframe

In [1]:
import pandas as pd

In [2]:
# Read the json file
df=pd.read_json('/content/cchhl_all.jsonl', lines=True)

In [3]:
# Familiar with database
df.head()
# This json file have two column need to be flatten: Search_result & record

,reference_id,search_result,record
0,1,"{'publisher': '', 'reference_title': 'Adapting...","{'Title': 'Adapting to climate change', 'Autho..."
1,2,"{'publisher': '', 'reference_title': 'Vector-b...",{'Title': 'Vector-borne diseases: Impact of cl...
2,3,"{'publisher': '', 'reference_title': 'Plan for...",{'Title': 'Plan for disaster before it happens...
3,4,"{'publisher': '', 'reference_title': 'Health i...","{'Title': 'Health impacts of climate change', ..."
4,5,"{'publisher': 'Routledge', 'reference_title': ...","{'Title': 'Climate change and adaptation', 'Au..."


### 2. Flatten two column

In [4]:
# First, flatten search result column
search_results_df = pd.json_normalize(df['search_result'])
# check the flatten column content
print(search_results_df.iloc[0])

publisher                                    
reference_title    Adapting to climate change
series_editor                                
reference_type                Journal Article
journal                            The Lancet
num                                       943
volume                                    371
year_published                           2008
author                                       
series_title                                 
geo                                         3
issue                                    9613
editor                                       
conference                                   
place_published                              
skip                                      943
geo_nonus                                   5
page_s                                    624
university                                   
book_title                                   
Name: 0, dtype: object


In [5]:
# Second, flatten record column
record_df = pd.json_normalize(df['record'])
# check the flatten column content
print(record_df.iloc[10])

Title                                                            A Commission on climate change
Author(s)                                                                     No authors listed
Year                                                                                       2009
Journal                                                            The Lancet. 373 (9676): 1659
Source                                        <ul class="cchh-content_detail-plainlist"> <li...
Exposure                                                                                    NaN
Geographic Feature                                        <ul> General Geographic Feature </ul>
Geographic Location                                   <ul> Global or Unspecified Location </ul>
Health Impact                                                  <ul> General Health Impact </ul>
Resource Type                                                     <ul> Commentary/Opinion </ul>
Special Topic                           

In [6]:
# Third, combine flatten result
result_df_1 = pd.concat([df.drop('search_result', axis=1), search_results_df], axis=1)
result_df = pd.concat([result_df_1, record_df], axis=1)
result_df.drop('record', axis=1, inplace=True)
print(result_df.iloc[0])
# In the combine dataset, there are some HTML column need to transform and there are some depulicate column

reference_id                                                                                  1
publisher                                                                                      
reference_title                                                      Adapting to climate change
series_editor                                                                                  
reference_type                                                                  Journal Article
journal                                                                              The Lancet
num                                                                                         943
volume                                                                                      371
year_published                                                                             2008
author                                                                                         
series_title                            

### 3. Export csv for dropping duplicated column

In [7]:
result_df.to_csv('cchhl_all_flatten.csv', index=False)

### 4. Drop Duplicate column

In the check code

In [12]:
# read drop duplicate column file
df=pd.read_csv('/content/cchhl_all_flatten_cleanduplicate.csv')

<ipython-input-12-b9e60a4b9da1>:2: DtypeWarning: Columns (2,9,10,14,15,26,27,28,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/cchhl_all_flatten_cleanduplicate.csv')


### 5. Transform HTML column

In [14]:
# flatten HTML column
import pandas as pd
from bs4 import BeautifulSoup

def extract_list_items(html_content):
    if pd.isna(html_content) or html_content is None:
        return ""

    soup = BeautifulSoup(html_content, 'html.parser')
    # replace all the c
    items = [li.get_text(strip=True).replace(',', ' |') for li in soup.find_all('li')]

    return ' |'.join(items)

for col in ['Source', 'Exposure', 'Geographic Feature', 'Geographic Location',
            'Health Impact', 'Resource Type', 'Special Topic', 'Model/Methodology'
            'Model Timescale','Climate Change and Socioeconomic Scenarios']:
    if col in df.columns:
        df[f'{col}'] = df[col].apply(extract_list_items)

### 6. Clean HTML Column

6.1. Clean Source Column

In [39]:
# Using this code to check each column
# check source column
for index, value in df['Source'].head(1000).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 0, Value: Publisherhttp://dx.doi.org/10.1016/S0140-6736(08)60276-7
Index: 1, Value: Publisherhttp://dx.doi.org/10.1007/s00436-008-1199-6
Index: 2, Value: Publisherhttp://dx.doi.org/10.1108/02580540810854677
Index: 4, Value: Publisherhttp://www.routledge.com/books/details/9781844076895/
Index: 5, Value: Publisherhttp://www.routledge.com/books/details/9781844076888/
Index: 6, Value: Publisherhttp://www.oecd.org/env/cc/economicaspectsofadaptationtoclimatechangecostsbenefitsandpolicyinstruments.htm
Index: 7, Value: Publisherhttp://www.euro.who.int/__data/assets/pdf_file/0006/95919/E91347.pdf
Index: 8, Value: Publisherhttp://www.euro.who.int/en/health-topics/environment-and-health/Climate-change/publications/pre-2009/protecting-health-in-europe-from-climate-change
Index: 9, Value: Publisherhttp://www.eurannallergyimm.com/cont/journals-articles/169/volume-news.asp
Index: 10, Value: Publisherhttp://dx.doi.org/10.1016/s0140-6736(09)60922-3
Index: 11, Value: Publisherhttp://www.muthar-al

In [40]:
df['Publisher1'] = ''
df['Publisher2'] = ''
df['PubMed'] = ''

for idx, row in df.iterrows():
    if pd.isna(row['Source']):
        continue

    parts = row['Source'].split('|')
    publisher_parts = [part.replace('Publisher', '').strip() for part in parts if 'Publisher' in part]
    pubmed_parts = [part.replace('PubMed', '').strip() for part in parts if 'PubMed' in part]

    if len(publisher_parts) >= 1:
        df.at[idx, 'Publisher1'] = publisher_parts[0]
    if len(publisher_parts) >= 2:
        df.at[idx, 'Publisher2'] = publisher_parts[1]

    if len(pubmed_parts) >= 1:
        df.at[idx, 'PubMed'] = pubmed_parts[0]

In [44]:
df.drop('Source', axis=1, inplace=True)

6.2. Exposure

In [57]:
for index, value in df['Exposure'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 0, Value: Extreme Weather-Related Event or Disaster | Temperature: Flood |Extreme Weather-Related Event or Disaster | Temperature: Heat
Index: 1, Value: Food Quality: Crop/Plant Pathogen
Index: 3, Value: Extreme Weather-Related Event or Disaster | Temperature: Drought | Flood |Extreme Weather-Related Event or Disaster | Temperature: Heat | Variability
Index: 6, Value: Ecosystem Change | Extreme Weather-Related Event or Disaster | Food Quality | Food Security | Glacier Melt/Snow Melt | Precipitation | Sea Level Rise | Temperature | Water Quality | Water Security: Drought | Flood | Hurricane | Landslide |Ecosystem Change | Extreme Weather-Related Event or Disaster | Food Quality | Food Security | Glacier Melt/Snow Melt | Precipitation | Sea Level Rise | Temperature | Water Quality | Water Security: Nutritional quality |Ecosystem Change | Extreme Weather-Related Event or Disaster | Food Quality | Food Security | Glacier Melt/Snow Melt | Precipitation | Sea Level Rise | Temperature 

6.3. Geographic Feature

In [56]:
for index, value in df['Geographic Feature'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 6, Value: Forest | Ocean/Coastal | Other Geographic Feature | Specify: Forests


6.4. Geographic Location

In [58]:
for index, value in df['Geographic Location'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 0, Value: Non-United States: Europe
Index: 1, Value: Non-United States: Africa | Asia | Europe
Index: 8, Value: Non-United States: Europe


6.5. Health Impact

In [59]:
for index, value in df['Health Impact'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 0, Value: Cancer | Dermatological Impact | Infectious Disease | Morbidity/Mortality: Foodborne DiseaseFoodborne Disease: General Foodborne DiseaseFoodborne Disease |Foodborne Disease: General Foodborne Disease
Index: 1, Value: Infectious Disease: Vectorborne Disease | Zoonotic DiseaseVectorborne Disease | Zoonotic Disease: Mosquito-borne Disease | Tick-borne DiseaseMosquito-borne Disease | Tick-borne Disease: Chikungunya | Malaria | West Nile Virus | Yellow FeverMosquito-borne DiseaseVectorborne Disease | Zoonotic Disease: Other Zoonotic Disease | SpecifyOther Zoonotic Disease | Specify: BartonellaZoonotic Disease (other)Vectorborne DiseaseZoonotic Disease |Vectorborne Disease | Zoonotic Disease: Mosquito-borne Disease | Tick-borne DiseaseMosquito-borne Disease | Tick-borne Disease: Chikungunya | Malaria | West Nile Virus | Yellow FeverMosquito-borne Disease |Mosquito-borne Disease | Tick-borne Disease: Chikungunya | Malaria | West Nile Virus | Yellow Fever |Vectorborne Disease 

6.6. Resource column noting include

In [79]:
df['Resource Type'].value_counts()

,count
Resource Type,
,22695


6.7. Special Topic

In [61]:
for index, value in df['Health Impact'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 0, Value: Cancer | Dermatological Impact | Infectious Disease | Morbidity/Mortality: Foodborne DiseaseFoodborne Disease: General Foodborne DiseaseFoodborne Disease |Foodborne Disease: General Foodborne Disease
Index: 1, Value: Infectious Disease: Vectorborne Disease | Zoonotic DiseaseVectorborne Disease | Zoonotic Disease: Mosquito-borne Disease | Tick-borne DiseaseMosquito-borne Disease | Tick-borne Disease: Chikungunya | Malaria | West Nile Virus | Yellow FeverMosquito-borne DiseaseVectorborne Disease | Zoonotic Disease: Other Zoonotic Disease | SpecifyOther Zoonotic Disease | Specify: BartonellaZoonotic Disease (other)Vectorborne DiseaseZoonotic Disease |Vectorborne Disease | Zoonotic Disease: Mosquito-borne Disease | Tick-borne DiseaseMosquito-borne Disease | Tick-borne Disease: Chikungunya | Malaria | West Nile Virus | Yellow FeverMosquito-borne Disease |Mosquito-borne Disease | Tick-borne Disease: Chikungunya | Malaria | West Nile Virus | Yellow Fever |Vectorborne Disease 

6.8. Abstract

In [62]:
for index, value in df['Abstract'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 2, Value: <p>Purpose - This paper aims to review the latest management developments across the globe and pinpoint practical implications from cutting-edge research and case studies. Design/methodology/approach - This briefing is prepared by an independent writer, who adds their own impartial comments and places the articles in context. Findings - As cyclones, hurricanes, floods and fires wreak havoc with communities round the world, the warnings that global warming might make the weather even worse and less predictable are at last being recognized by even the most stubborn of "head-in-the-sand" optimists. So how do you stay in business if you are flooded out, burned down or blown away? Surely you have a plan? Practical implications - The paper provides strategic insights and practical thinking that have influenced some of the world's leading organizations. Originality/value - The briefing saves busy executives and researchers hours of reading time by selecting only the very best

In [63]:
# remove the <p> tag and any other formatting
df['Abstract']=df['Abstract'].str.replace('<p>', ' ')

6.9. Clean Model/Methodology column

In [64]:
for index, value in df['Model/Methodology'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 6, Value: <ul> Cost/Economic Impact Prediction, Methodology </ul>


In [65]:
# flattened out with pipe

def extract_items(html_content):
    if pd.isna(html_content) or html_content is None:
        return ""

    # First
    soup = BeautifulSoup(html_content, 'html.parser')
    items= soup.get_text(strip=True).replace(',', ' |')

    return items

df['Model/Methodology']=df['Model/Methodology'].apply(extract_items)

In [67]:
for index, value in df['Model/Methodology'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 6, Value: Cost/Economic Impact Prediction | Methodology


6.10. Clean Model Timescale

In [68]:
for index, value in df['Model Timescale'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 6, Value: <ul> Long-Term (&gt;10 years) </ul>


In [70]:
df['Model Timescale']=df['Model Timescale'].apply(extract_items)
for index, value in df['Model Timescale'].head(10).items():
    if isinstance(value, str) and value.strip() != '':
        print(f"Index: {index}, Value: {value}")

Index: 6, Value: Long-Term (>10 years)


6.11. Climate change and socioeconomics scenarios

In [77]:
df['Climate Change and Socioeconomic Scenarios'].value_counts()

,count
Climate Change and Socioeconomic Scenarios,
,22563
Representative Concentration Pathway (RCP) | Other Climate Change Scenario | Specify: CMIP5,3
Other Climate Change Scenario | Specify: Ecological Niche Model,2
Other Climate Change Scenario | Specify: Business As Usual,2
Other Climate Change Scenario | Specify: Unspecified IPCC CC Scenario,2
...,...
Other Climate Change Scenario | Specify: IPCC WGIII AR5 Scenarios,1
Other Climate Change Scenario | Specify: Mean Annual Temperature,1
Other Socioeconomic Scenario | Specify: Unspecified Low Carbon Scenario,1


### 7. Rename column

In [80]:
# Drop unnecessary column
drop_columns=['skip', 'university']
for col in drop_columns:
  df.drop(col, axis=1, inplace=True)

In [83]:
df.dtypes

,0
reference_id,int64
reference_title,object
series_editor,object
reference_type,object
num,int64
volume_detailed,object
year_published,int64
geo,int64
issue,object
conference,object


In [84]:
df = df.rename(columns={
    'Model/Methodology': 'Model_Methodology',
    'Author(s)': 'Author',
    'Publisher1': 'Publisher1_URL',
    'Publisher2': 'Publisher2_URL',
    'PubMed': 'PubMed_URL',
    'Geographic Feature': 'Geographic_Features',
    'Geographic Location': 'Geographic_Location',
    'Health Impact': 'Health_Impact',
    'Resource Type' : 'Resource_Type',
    'Special Topic' : 'Special_Topic',
    'Climate Change and Socioeconomic Scenarios' : 'Climate_Change_and_Socioeconomic_Scenarios',
    'Model Timescale' : 'Model_Timescale'
})

### 8. Change year column type

In [86]:
# Year field: should be saved an an integer, not a decimal
df['year_published'] = df['year_published'].apply(pd.to_numeric, errors='coerce')

### 9. Write out a new CSV file AND a new sqlite database file

In [88]:
df.to_csv('cchhl.csv', index=False)

In [90]:
import sqlite3
conn = sqlite3.connect('cchhl.db')
df.to_sql('cchhl', conn, if_exists='replace', index=False)

22695